# Criar VENV

https://oandersonbm.medium.com/usando-python-no-vs-code-com-ambiente-virtual-venv-ecef7959b652

In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import locale

# Definindo a configuração local para o Brasil (pt_BR)
locale.setlocale(locale.LC_ALL, 'pt_BR')

# Personalizando para ver todas as colunas do DataFrame
pd.set_option('display.max_columns', None)

# Base de dados

In [6]:
# Realizando a junção dos 3 arquivos .CSV num único DataFrame

# Caminhos para os arquivos .CSV
caminho_1 = r"base_dados\Cel Manfrini.csv"

caminho_2 = r"base_dados\Cel Manfrini(1).csv"

caminho_3 = r"base_dados\Cel Manfrini(2).csv"

caminho_4 = r"base_dados\Cel Manfrini(3).csv"


# Leitura dos arquivos .CSV em DataFrames individuais
df1 = pd.read_csv(caminho_1)
df2 = pd.read_csv(caminho_2)
df3 = pd.read_csv(caminho_3)
df4 = pd.read_csv(caminho_4)

# Concatenação dos DataFrames em um único DataFrame
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Excluindo linhas que contenham a string "Tela" em qualquer coluna
df = df[~df.apply(lambda row: row.astype(str).str.contains('Tela')).any(axis=1)]

# Transformando o tipo de dado das colunas de valores monetários
df['A_LIQUIDAR'] = df['A_LIQUIDAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['LIQUIDADO_A_PAGAR'] = df['LIQUIDADO_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['TOTAL_A_PAGAR'] = df['TOTAL_A_PAGAR'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df['PAGO'] = df['PAGO'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))

# Convertendo a coluna DATA para datetime
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%y')

# Convertendo a coluna DIAS em número inteiro
df['DIAS'] = df['DIAS'].astype(int)



# Trabalhando com um DATAFRAME voltado para controle dos SALDOS

In [7]:
df_saldos = df[['SIGLA UG', 'ANO', 'NOME_CREDOR', 'DATA', 'DIAS', 'ND','NDSI', 'NOME_NDSI', 'NE', 'PI', 'NOME_PI','TIPO', 'NOME_UGR', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'TOTAL_A_PAGAR', 'PAGO']]

In [8]:
# Saldos Totais
total_pago = df['PAGO'].sum().round(2)
total_a_liquidar = df['A_LIQUIDAR'].sum().round(2)
total_liquidado_a_pagar = df['LIQUIDADO_A_PAGAR'].sum().round(2)
total_a_pagar = df['TOTAL_A_PAGAR'].sum().round(2)


print(f'O total pago pela ESA até o momento foi: {locale.currency(total_pago, grouping=True)}')
print(f'O total de saldo a liquidar pela ESA até o momento foi: {locale.currency(total_a_liquidar, grouping=True)}')
print(f'O total de saldo liquidado a pagar pela ESA até o momento foi: {locale.currency(total_liquidado_a_pagar, grouping=True)}')
print(f'O total de saldo a pagar pela ESA até o momento foi: {locale.currency(total_a_pagar, grouping=True)}')

O total pago pela ESA até o momento foi: R$ 23.713.415,99
O total de saldo a liquidar pela ESA até o momento foi: R$ 6.342.223,34
O total de saldo liquidado a pagar pela ESA até o momento foi: R$ 451.413,45
O total de saldo a pagar pela ESA até o momento foi: R$ 6.793.636,79


In [9]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ND', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ND',
             labels={'ND':'Natureza da Despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=500,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [10]:
# Saldo a liquidar por Subitem
cols = ['NDSI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NDSI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NDSI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por NDSI',
             labels={'NDSI':'Subitem da despesa', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=600,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [11]:
# Saldo a liquidar por tipo de PI
cols = ['NOME_PI', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_PI').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_PI', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por PI',
             labels={'NOME_PI':'Tipo de PI', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=800,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [12]:
# Saldo a liquidar por tipo de Empenho
cols = ['TIPO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por tipo de empenho',
             labels={'TIPO':'Tipo de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [13]:
# Saldo a liquidar por ano
cols = ['ANO', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('ANO').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='ANO', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'ANO':'Ano do Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [14]:
# Saldo a liquidar por tipo de NE
cols = ['TIPO_NE', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('TIPO_NE').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='TIPO_NE', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'TIPO_NE':'Tipo de Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [15]:
# Saldo a liquidar por tipo de NE
cols = ['NOME_UGR', 'A_LIQUIDAR']

df_aux = df.loc[:, cols].groupby('NOME_UGR').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
fig = px.bar(df_aux, 
             x='NOME_UGR', 
             y='A_LIQUIDAR',
             title='Saldo a liquidar por ano do empenho',
             labels={'NOME_UGR':'Nome das UGR', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             height=400,
             log_y=True,
             color='A_LIQUIDAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()

In [16]:
# Saldo a liquidar por ND
cols = ['ND', 'A_LIQUIDAR', 'LIQUIDADO_A_PAGAR', 'PAGO']

df_aux = df.loc[:, cols].groupby('ND').sum().sort_values(by= 'A_LIQUIDAR', ascending=False).reset_index()
df_aux = df_aux[df_aux['A_LIQUIDAR'] > 0]
# Criando o gráfico
fig = go.Figure()

# Adicionando as colunas y1, y2 e y3 no eixo y
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['A_LIQUIDAR'], name='A Liquidar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['LIQUIDADO_A_PAGAR'], name='Liquidado a pagar'))
fig.add_trace(go.Bar(x=df_aux['ND'], y=df_aux['PAGO'], name='Pago'))

fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")
fig.update_yaxes(type="log")

fig.show()

In [17]:
df[(df['ACAO'] == '212O') & df['A_LIQUIDAR'] != 0]

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
447,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-07-10,80,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000863,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,15000.00,0.0,15000.00,0.0
448,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-07-24,66,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000888,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,22455.00,0.0,22455.00,0.0
449,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-07-25,65,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000897,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,18600.00,0.0,18600.00,0.0
450,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-07-25,65,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000898,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,9000.00,0.0,9000.00,0.0
451,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-08,51,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000960,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,24950.00,0.0,24950.00,0.0
452,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-11,48,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000989,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,10800.00,0.0,10800.00,0.0
453,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-11,48,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE000990,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,28443.00,0.0,28443.00,0.0
454,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-16,43,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE001018,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,9900.00,0.0,9900.00,0.0
455,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-23,36,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE001059,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,14400.00,0.0,14400.00,0.0
456,160129,ESA,212O,MOVIMENTACAO DE MILITARES,2023,03849500000190,A. CENTRAL TRANSPORTES LTDA,2023-08-29,30,160533,4º CGCFEX,339033,PASSAGENS E DESPESAS COM LOCOMOCAO,33903304,MUDANCAS EM OBJETO DE SERVICO,2023NE001079,D6DAATSLOCO,ATS - ATIVIDADES SETORIAIS,6012,171389,ANO,ESTIMATIVO,ESCOLA DE SARGENTOS DAS ARMAS,PR,MG,160505,DGP,D6DA,ATS,LOCO,11160.00,0.0,11160.00,0.0


# Suporte para o Dashboard

In [18]:
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']

df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
df_aux

,NE,A_LIQUIDAR,DIAS,NOME_NDSI,NDSI
1059,2022NE000742,180.00,450,LOCACAO BENS MOV. OUT.NATUREZAS E INTANGIVEIS,33903914
1050,2022NE000788,6334.69,441,OUTSOURCING DE IMPRESSAO,33904016
1044,2022NE001087,3196.00,357,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
1077,2022NE001085,11086.68,357,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,33903917
1078,2022NE001086,6933.36,357,MANUTENCAO E CONSERVACAO DE EQUIPAMENTOS DE TIC,33904012
1067,2022NE002421,5758.75,302,SERVICOS GRAFICOS E EDITORIAIS,33903963
1299,2022NE001843,1016.25,302,SERVICOS GRAFICOS E EDITORIAIS,33903963
881,2022NE002483,283724.38,288,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
880,2022NE002482,157379.25,288,MANUTENCAO E CONSERV. DE BENS IMOVEIS,33903916
254,2022NE002691,11801.19,275,SERVICOS DE ENERGIA ELETRICA,33903943


In [19]:
# Saldo a liquidar por tipo de NE
cols = ['NE', 'A_LIQUIDAR', 'DIAS', 'NOME_NDSI', 'NDSI']
df_aux = df.loc[:, cols].sort_values(by='DIAS', ascending=False)
df_aux = df_aux[df_aux['A_LIQUIDAR'] != 0].head(10)
fig = px.bar(df_aux, 
             x='NE', 
             y='A_LIQUIDAR',
             title='10 empenhos com mais tempo na tela',
             labels={'NE':'Nota de Empenho', 'A_LIQUIDAR':'Saldo a liquidar'},
             text_auto='.3s',
             color='DIAS',
             color_continuous_scale = 'reds',
             log_y=True,
             template='plotly_dark',
             hover_name="NE", hover_data={'DIAS': True, 'A_LIQUIDAR': True, 'NOME_NDSI': True, 'NDSI': True})
fig.update_traces(textposition = 'outside')


fig.show()

## Informações para SideBar

In [20]:
# Qtde de empenhos realizados até o momento
df.shape[0]

3008

# Requisição em API para consulta a CNPJ

In [21]:
import requests
from pprint import pprint

url = "https://receitaws.com.br/v1/cnpj/21331404000138"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

pprint(response.json())

{'abertura': '03/11/2014',
 'atividade_principal': [{'code': '79.11-2-00', 'text': 'Agências de viagens'}],
 'atividades_secundarias': [{'code': '59.11-1-99',
                             'text': 'Atividades de produção cinematográfica, '
                                     'de vídeos e de programas de televisão '
                                     'não especificadas anteriormente'},
                            {'code': '63.99-2-00',
                             'text': 'Outras atividades de prestação de '
                                     'serviços de informação não especificadas '
                                     'anteriormente'},
                            {'code': '79.12-1-00',
                             'text': 'Operadores turísticos'},
                            {'code': '79.90-2-00',
                             'text': 'Serviços de reservas e outros serviços '
                                     'de turismo não especificados '
                                    

In [22]:
def consulta_cnpj(numero_cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{numero_cnpj}"

    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)
    
    dic_fornecedor = response.json()
    
    return dic_fornecedor

# Criar um dicionário vazio fora da função
dicionario = {}

# Chamar a função e atribuir o resultado ao dicionário
dicionario = consulta_cnpj('81243735001977')

# Imprimir o dicionário
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [23]:
pprint(dicionario)

{'abertura': '28/08/2015',
 'atividade_principal': [{'code': '26.21-3-00',
                          'text': 'Fabricação de equipamentos de informática'}],
 'atividades_secundarias': [{'code': '26.22-1-00',
                             'text': 'Fabricação de periféricos para '
                                     'equipamentos de informática'},
                            {'code': '27.59-7-99',
                             'text': 'Fabricação de outros aparelhos '
                                     'eletrodomésticos não especificados '
                                     'anteriormente, peças e acessórios'},
                            {'code': '46.19-2-00',
                             'text': 'Representantes comerciais e agentes do '
                                     'comércio de mercadorias em geral não '
                                     'especializado'},
                            {'code': '46.51-6-01',
                             'text': 'Comércio atacadista de equipam

In [24]:
dicionario['nome']

'POSITIVO TECNOLOGIA S.A.'

In [25]:
dicionario['telefone']

'(41) 3316-7700'

In [26]:
dicionario['email']

'fiscal@positivo.com.br'

In [27]:
dicionario['logradouro'] + ', ' + dicionario['numero'] + ' - ' + dicionario['bairro'] + ' - ' + dicionario['municipio'] + ' / ' + dicionario['uf']

'R JAVARI, 1255 - DISTRITO INDUSTRIAL I - MANAUS / AM'

In [28]:
# filtrando dos dados da coluna FAV
df_cnpj = df[df['NOME_PI'] != 'PROFISSIONAL DE SAUDE AUTONOMO']
df_cnpj = df[df['NOME_PI'] != 'ATENDIMENTO MEDICO-HOPITALR/FC']
df_cnpj = df[df['NOME_PI'] != 'IND-INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'IND - INDENIZACOES']
df_cnpj = df[df['NOME_PI'] != 'INDENIZACAO DESPESAS.']
df_cnpj = df[df['NOME_PI'] != 'AUXILIO FINANCEIRO NÃO INDENIZÁVEL']
df_cnpj = df[df['NOME_PI'] != 'INRE - RESTITUICAO E RESSARCIMENTO']
df_cnpj = df[df['NOME_NDSI'] != 'OUTROS SERV.DE TERCEIROS PJ- PAGTO ANTECIPADO']
df_cnpj = df[df['CREDOR'] != '160129']

df_cnpj['CREDOR'].unique()

array(['01590728000850', '05872523000141', '06981180000116',
       '09583781000169', '12742245000173', '17281106000103',
       '20584944000160', '27325275000151', '27995686000154',
       '34028316001509', '39319890000134', '41505690000118',
       '41776201000162', '43003732000148', '43433811000199',
       '44812602000119', '45424853000199', '76535764000143',
       '00139736000145', '01255291000121', '01804159000121',
       '02338743000100', '02347121000210', '03157268000120',
       '03539933000140', '04785664000164', '05088041000103',
       '06022475000165', '06894510000136', '07027329743',
       '07457410000296', '07912053000128', '08605233000120',
       '08703140000138', '09249668000141', '09505477000101',
       '10463704000154', '10526703000101', '10842829000195',
       '10902067000175', '11094173000132', '11532912000120',
       '12669334000131', '13857945000176', '13986656000177',
       '14459158000139', '14807420000199', '14951451000119',
       '14968227000130', '1

In [29]:
df_cnpj[df_cnpj['CREDOR'] == '160129']

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO


In [30]:
ano_atual = str(datetime.now().year)


df_corrente = df[df['ANO'] == ano_atual]
df_corrente

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
31,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,00139736000145,TECELAGEM SAO DOMINGOS LTDA,2023-09-12,16,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001136,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,450.0,0.0,450.0,0.0
32,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,01255291000121,H.A DISTRIBUIDORA LTDA,2023-04-11,170,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000361,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",MG,MG,160073,DGO,I3DA,FUN,ADOM,0.0,0.0,0.0,2646.0
33,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,01255291000121,H.A DISTRIBUIDORA LTDA,2023-06-22,98,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000737,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,0.0,0.0,0.0,4647.0
34,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,01255291000121,H.A DISTRIBUIDORA LTDA,2023-07-31,59,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE000930,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,0.0,0.0,0.0,2211.2
35,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2023,01255291000121,H.A DISTRIBUIDORA LTDA,2023-09-12,16,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2023NE001137,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3DA,FUN,ADOM,541.0,0.0,541.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,59717553000617,MULTILASER INDUSTRIAL S.A.,2023-05-26,125,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE001046,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,"ESCOLA DE SARGENTOS DAS ARMAS, AVENIDA SETE DE...",MG,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.0,0.0,0.0,690.0
3007,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,65095531000150,LOGOS PAPELARIA LTDA,2023-05-25,126,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905245,EQUIPAMENTOS DE TIC - IMPRESSORAS,2023NE001044,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.0,0.0,0.0,1800.0
3008,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,81243735000903,POSITIVO TECNOLOGIA S.A.,2023-02-15,225,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2023NE000261,E3PCFSCINFO,INFORMATICA,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,BA,MG,167504,COEX - GESTOR,E3PC,FSC,INFO,0.0,0.0,0.0,7114.0
3009,167129,ESA,2919,REGISTRO E FISCALIZACAO DE PRODUTOS C,2023,98488040644,MARIO HENRIQUE CUBBI,2023-05-22,129,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903001,COMBUSTIVEIS E LUBRIFICANTES AUTOMOTIVOS,2023NE001035,E3PCFSCCOLU,COMBUSTIVEIS E LUBRIFICANTES,6012,215839,ANO,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,167504,COEX - GESTOR,E3PC,FSC,COLU,0.0,0.0,0.0,2236.2


In [31]:
df['ANO'].unique()

array(['2022', '2023', '2021'], dtype=object)

# Pesquisas no DataFrame

In [32]:
# Criando um DataFrame para testes 
df_teste = df.copy()

df_teste.head(10)

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,01590728000850,MICROTECNICA INFORMATICA LTDA,2022-11-03,329,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2022NE001245,I3FPPREDEGE,DESPESAS GERAIS DAS UGE - GESTÃO EME.,0032,171460,RPNP,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,MG,MG,160073,DGO,I3FP,PRE,DEGE,0.00,0.0,0.00,14637.78
1,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,01590728000850,MICROTECNICA INFORMATICA LTDA,2022-12-05,297,160533,4º CGCFEX,449052,EQUIPAMENTOS E MATERIAL PERMANENTE,44905241,EQUIPAMENTOS DE TIC - COMPUTADORES,2022NE002430,IXAPFUNDGOA,ADMINISTRACAO DAS OM,0032,171502,RPNP,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160507,EME-GESTOR,IXAP,FUN,DGOA,0.00,0.0,0.00,21956.67
2,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,05872523000141,AEF - MANUTENCAO EM EQUIPAMENTOS INDUSTRIAIS LTDA,2022-04-12,534,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2022NE000344,I3DACNTMABM,MNT DE MÁQ E EQP ADMINISTRATIVOS,0032,171460,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160073,DGO,I3DA,CNT,MABM,0.00,0.0,0.00,7480.83
3,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,05872523000141,AEF - MANUTENCAO EM EQUIPAMENTOS INDUSTRIAIS LTDA,2022-06-09,476,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2022NE000601,I3DACNTMABM,MNT DE MÁQ E EQP ADMINISTRATIVOS,0032,171460,RPP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160073,DGO,I3DA,CNT,MABM,0.00,0.0,0.00,6780.83
4,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,05872523000141,AEF - MANUTENCAO EM EQUIPAMENTOS INDUSTRIAIS LTDA,2022-10-27,336,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2022NE001220,I3DACNTMABM,MNT DE MÁQ E EQP ADMINISTRATIVOS,0032,203677,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160073,DGO,I3DA,CNT,MABM,0.00,0.0,0.00,3238.56
5,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,05872523000141,AEF - MANUTENCAO EM EQUIPAMENTOS INDUSTRIAIS LTDA,2022-10-27,336,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2022NE001220,I3DACNTMABM,MNT DE MÁQ E EQP ADMINISTRATIVOS,0032,203677,RPP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160073,DGO,I3DA,CNT,MABM,0.00,0.0,0.00,700.00
6,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,05872523000141,AEF - MANUTENCAO EM EQUIPAMENTOS INDUSTRIAIS LTDA,2022-12-29,273,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903917,MANUT. E CONSERV. DE MAQUINAS E EQUIPAMENTOS,2022NE002723,I3DACNTMABM,MNT DE MÁQ E EQP ADMINISTRATIVOS,0032,171460,RPNP,GLOBAL,ESCOLA DE SARGENTOS DAS ARMAS,MG,MG,160073,DGO,I3DA,CNT,MABM,809.64,0.0,809.64,39023.43
7,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,06981180000116,CEMIG DISTRIBUICAO S.A,2022-06-14,471,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903943,SERVICOS DE ENERGIA ELETRICA,2022NE000619,I3DACSPENEL,DESPESAS COM ENERGIA ELETRICA,0032,203677,RPNP,ESTIMATIVO,0,MG,MG,160073,DGO,I3DA,CSP,ENEL,0.00,0.0,0.00,254563.30
8,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,09583781000169,2 CLICK SOLUCOES LTDA,2022-11-11,321,160533,4º CGCFEX,339030,MATERIAL DE CONSUMO,33903022,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZACAO,2022NE001692,I3DAFUNADOM,ADMINISTRACAO DE OM,0032,171460,RPNP,ORDINARIO,ESCOLA DE SARGENTOS DAS ARMAS - AVENIDA SETE ...,GO,MG,160073,DGO,I3DA,FUN,ADOM,0.00,0.0,0.00,1152.92
9,160129,ESA,2000,ADMINISTRACAO DA UNIDADE,2022,12742245000173,UNISERVE COMERCIO E SERVICOS TERCEIRIZADOS LTDA,2022-06-09,476,160533,4º CGCFEX,339037,LOCACAO 

In [33]:
cols = ['NOME_NDSI', 'A_LIQUIDAR']

df_aux = df_teste.loc[:,cols].groupby(by='NOME_NDSI').sum().sort_values(by='A_LIQUIDAR', ascending=False).reset_index()


df_aux.head(10)

,NOME_NDSI,A_LIQUIDAR
0,MANUTENCAO E CONSERV. DE BENS IMOVEIS,872557.73
1,SERVICOS DE ENERGIA ELETRICA,737421.19
2,GENEROS DE ALIMENTACAO,686598.41
3,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",518085.54
4,DIARIAS NO PAIS,344601.14
5,MUDANCAS EM OBJETO DE SERVICO,285868.00
6,SERVICOS GRAFICOS E EDITORIAIS,259826.32
7,LIMPEZA E CONSERVACAO,256692.09
8,OUTSOURCING DE IMPRESSAO,204618.28
9,ALIMENTOS PARA ANIMAIS,199424.38


## Empenhos com mais de 100 dias

### Empenhos da ND 30

In [34]:
# Empenhos da ND 30 com prazo de entrega superior a 100 dias

df_nd30_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '339030') & (df_teste['DIAS'] > 100) & (df_teste['TIPO_NE'] == 'ORDINARIO') ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd30_100dias.sort_values(by='A_LIQUIDAR', ascending=False).reset_index(drop=True)

df_nd30_100dias.to_excel('nd30_PA.xlsx')

### Empenhos ND 52

In [35]:
df_nd52_100dias = df_teste[(df_teste['A_LIQUIDAR'] != 0) & (df_teste['ND'] == '449052') & (df_teste['DIAS'] > 50) ].sort_values(by='DIAS', ascending=False).reset_index(drop=True)

df_nd52_100dias.reset_index(drop=True)

df_nd52_100dias.to_excel('nd52_PA.xlsx')

### Recursos Liquidados a pagar

In [36]:
df_liqui_a_pagar =df_teste[ df_teste['LIQUIDADO_A_PAGAR'] != 0 ]

df_liqui_a_pagar.sort_values(by='LIQUIDADO_A_PAGAR', ascending=False).reset_index(drop=True).head(2)

,UG,SIGLA UG,ACAO,NOME_ACAO,ANO,CREDOR,NOME_CREDOR,DATA,DIAS,ICFEX,NOME_ICFEX,ND,NOME_ND,NDSI,NOME_NDSI,NE,PI,NOME_PI,PROGRAMA,PTRES,TIPO,TIPO_NE,LOCAL,UF_CREDOR,UF_UG,UGR,NOME_UGR,RADPI4,MEIPI3,SUFPI4,A_LIQUIDAR,LIQUIDADO_A_PAGAR,TOTAL_A_PAGAR,PAGO
0,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,05741540000140,ONCOMINAS ONCOLOGIA CLINICA LTDA,2023-08-29,30,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903950,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",2023NE001460,D8SAFUSOCSA,ORGANIZACAO CIVIL DE SAUDE,0032,215845,ANO,ESTIMATIVO,CLINICA/CONSULTÓRIO/HOSPITAL,MG,MG,167505,DGP,D8SA,FUS,OCSA,0.0,144088.36,144088.36,0.0
1,167129,ESA,2004,ASSISTENCIA MEDICA E ODONTOLOGICA AOS,2023,42855999000362,UNIMED TRES CORACOES COOPERATIVA DE TRAB MEDIC...,2023-08-28,31,160533,4º CGCFEX,339039,OUTROS SERVICOS DE TERCEIROS - PESSOA JURIDICA,33903950,"SERV.MEDICO-HOSPITAL.,ODONTOL.E LABORATORIAIS",2023NE001422,D8SAFUSOCSA,ORGANIZACAO CIVIL DE SAUDE,0032,215845,ANO,ESTIMATIVO,CLINICA/CONSULTÓRIO/HOSPITAL,MG,MG,167505,DGP,D8SA,FUS,OCSA,0.0,42863.15,42863.15,0.0


In [37]:
total_liqui_a_pagar = df_liqui_a_pagar['LIQUIDADO_A_PAGAR'].sum()
locale.currency(total_liqui_a_pagar, grouping=True)

'R$ 451.413,45'

In [38]:
df_aux = df_liqui_a_pagar.groupby(by='NOME_NDSI')['LIQUIDADO_A_PAGAR'].sum().sort_values( ascending=False).reset_index()

fig = px.bar(df_aux,
             x = 'NOME_NDSI',
             y = 'LIQUIDADO_A_PAGAR',
             log_y=True,
             height=900,
             title='Saldos liquidados a pagar por NDSI',
             labels={'NOME_NDSI':'Descrição do Subitem', 'LIQUIDADO_A_PAGAR':'Liquidado a Pagar'},
             text_auto='.3s',
             color='LIQUIDADO_A_PAGAR',
             color_continuous_scale = 'reds',
             template='plotly_dark')
fig.update_traces(textposition = 'outside', cliponaxis = False, hovertemplate=None)
fig.update_layout(hovermode="x unified")

fig.show()